this notebook will be evaluating the fairness metrics, and the meat of it all, the Bias Audit. 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Fairlearn imports

from fairlearn.metrics import (
    MetricFrame,
    demographic_parity_difference,
    demographic_parity_ratio,
    equalized_odds_difference,
    selection_rate
)

# Load and prep
df = pd.read_csv('../data/adult_clean.csv')
X = df.drop(columns=['class'])
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# preprocessing
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(drop='first',handle_unknown='ignore'), categorical_cols)
    ]
)   

# Train
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Model ready. Starting fairness audit.")


Model ready. Starting fairness audit.


In [5]:
y_test_binary = (y_test == '>50K').astype(int)
y_pred_binary = (y_pred == '>50K').astype(int)

print(f"y_test_binary sample: {y_test_binary[:10].values}")
print(f"y_pred_binary sample: {y_pred_binary[:10]}")

y_test_binary sample: [0 1 0 1 0 1 0 1 0 0]
y_pred_binary sample: [0 0 0 1 0 0 0 0 0 0]


In [6]:
# get the sensitive attribute from test set
sex_test = X_test['sex']

# MetricFrame allows us to compute metrics across groups
metric_frame_sex = MetricFrame(
    metrics={'selection_rate': selection_rate},
        y_true=y_test_binary,
        y_pred=y_pred_binary,
        sensitive_features=sex_test
)

print("Selection Rate By Sex:")
print(metric_frame_sex.by_group)
print(f"\nOverall: {metric_frame_sex.overall['selection_rate']:.4f}")

Selection Rate By Sex:
        selection_rate
sex                   
Female        0.077108
Male          0.260976

Overall: 0.2017


In [7]:
dp_diff = demographic_parity_difference(
    y_test_binary,
    y_pred_binary,
    sensitive_features=sex_test
)

dp_ratio = demographic_parity_ratio(
    y_test_binary,
    y_pred_binary,
    sensitive_features=sex_test
)

print(f"Demographic Parity Difference: {dp_diff:.4f}")
print(f"Demographic Parity Ratio: {dp_ratio:.4f}")

Demographic Parity Difference: 0.1839
Demographic Parity Ratio: 0.2955


In [8]:
race_test = X_test['race']

dp_diff_race = demographic_parity_difference(
    y_test_binary,
    y_pred_binary,
    sensitive_features=race_test
)

dp_ratio_race = demographic_parity_ratio(
    y_test_binary,
    y_pred_binary,
    sensitive_features=race_test
)

print(f"Race - Demographic Parity Difference: {dp_diff_race:.4f}")
print(f"Race - Demographic Parity Ratio: {dp_ratio_race:.4f}")

Race - Demographic Parity Difference: 0.2204
Race - Demographic Parity Ratio: 0.2209


In [9]:
metric_frame_race = MetricFrame(
    metrics={'selection_rate': selection_rate},
        y_true=y_test_binary,
        y_pred=y_pred_binary,
        sensitive_features=race_test
)

print("Selection Rate by Race:")
print(metric_frame_race.by_group.sort_values(by='selection_rate'))

Selection Rate by Race:
                    selection_rate
race                              
Other                     0.062500
Amer-Indian-Eskimo        0.088889
Black                     0.091885
White                     0.213524
Asian-Pac-Islander        0.282946


In [10]:
from fairlearn.metrics import false_positive_rate, true_positive_rate

metric_frame_sex_full = MetricFrame(
    metrics={
        'selection_rate': selection_rate,
        'true_positive_rate': true_positive_rate,
        'false_positive_rate': false_positive_rate
    },
    y_true=y_test_binary,
    y_pred=y_pred_binary,
    sensitive_features=sex_test
)

print('Full Metrics By Sex:')
print(metric_frame_sex_full.by_group)

Full Metrics By Sex:
        selection_rate  true_positive_rate  false_positive_rate
sex                                                            
Female        0.077108            0.473829             0.020744
Male          0.260976            0.617350             0.103343
